In [1]:
import networkx as nx

In [25]:
g = nx.Graph()

In [26]:
g.add_node(0, a=5)

In [27]:
g.nodes

NodeView((0,))

In [28]:
g.edges

EdgeView([])

In [29]:
g.add_node(1, a=1)

In [30]:
g.add_edge(0, 1, weight=1.2, distance=100)

In [31]:
g.edges(data=True)

EdgeDataView([(0, 1, {'weight': 1.2, 'distance': 100})])

In [34]:
g.nodes(data=True)[0]['a']

5

In [16]:
def euclid_distance(x: float, y: float, x2: float, y2: float) -> float:
    return ((x - x2)**2 + (y - y2)**2)**0.5

In [59]:
def read_graph(file_path: str) -> nx.Graph:
    g = nx.Graph()
    with open(file_path, 'r') as f:
        for line in f:
            if line[0].isdigit():
                parts = line.split()
                v = int(parts[0])
                v_x = float(parts[1])
                v_y = float(parts[2])
                g.add_node(v, x=v_x, y=v_y)
                for u in g.nodes:
                    if u == v:
                        continue
                    u_x = g.nodes(data=True)[u]['x']
                    u_y = g.nodes(data=True)[u]['y']
                    g.add_edge(u, v, weight=euclid_distance(u_x, u_y, v_x, v_y), pheromones=1)
        return g

In [60]:
g = read_graph('../2024_2025/live/dj38.tsp')

In [61]:
g.nodes

NodeView((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38))

In [62]:
g.edges

EdgeView([(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 28), (3, 29), (3, 30), (3, 31), (3, 32), (3, 33), (3, 34), (3, 35), (3, 36), (3, 37), (3, 38), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9),

In [63]:
g.edges[1,2]

{'weight': 290.99301545433866, 'pheromones': 1}

In [64]:
import numpy as np

In [99]:
def traverse(g: nx.Graph, start_node: int, alpha: float = 1, beta: float = 1):
    cycle = [start_node]
    cycle_weight = 0
    visited = {start_node}
    curr_node = start_node
    while len(visited) < len(g.nodes):
        unvisited_neighbors = list(set(g[curr_node]) - visited)
        weights = np.array([g[curr_node][u]['weight'] for u in unvisited_neighbors])
        pheromones = np.array([g[curr_node][u]['pheromones'] for u in unvisited_neighbors])
        probs_weighted = pheromones**alpha / weights**beta
        probs = probs_weighted / probs_weighted.sum()
        next_node = int(np.random.choice(unvisited_neighbors, size=1, p=probs)[0])

        cycle.append(next_node)
        cycle_weight += g[curr_node][next_node]['weight']
        curr_node = next_node
        visited.add(next_node)
    
    cycle_weight += g[curr_node][start_node]['weight']
    cycle.append(start_node)
    return cycle, cycle_weight

In [100]:
int(np.random.choice([1,2,3], size=1)[0])

2

In [101]:
# import random
# random.choices()

In [102]:
traverse(g, 1)

([1,
  14,
  10,
  21,
  29,
  22,
  26,
  25,
  23,
  20,
  24,
  13,
  15,
  7,
  5,
  3,
  4,
  35,
  37,
  38,
  28,
  27,
  31,
  36,
  33,
  34,
  17,
  18,
  19,
  16,
  12,
  11,
  8,
  9,
  6,
  2,
  32,
  30,
  1],
 11401.959180986878)

In [103]:
def aco(g: nx.Graph, num_iters: int, num_ants: int, start_node: int = 1, evaporation_rate: float = 0.9):
    best_cycle, best_cycle_weight = None, float('inf')
    for i in range(num_iters):
        cycles = []
        for ant in range(num_ants):
            cycle, cycle_weight = traverse(g, start_node=start_node)
            cycles.append((cycle, cycle_weight))

        for cycle, cycle_weight in cycles:
            for u, v in zip(cycle[:-1], cycle[1:]):
                g.edges[u,v]['pheromones'] += 1 / cycle_weight
            
            if cycle_weight < best_cycle_weight:
                best_cycle_weight = cycle_weight
                best_cycle = cycle.copy()

        for edge in g.edges:
            g.edges[edge]['pheromones'] *= evaporation_rate

    return best_cycle, best_cycle_weight

In [107]:
aco(g, num_iters=100, num_ants=50, evaporation_rate=0.95)

([1,
  2,
  4,
  3,
  5,
  6,
  7,
  8,
  9,
  12,
  11,
  19,
  16,
  17,
  18,
  13,
  15,
  20,
  23,
  25,
  26,
  22,
  24,
  28,
  27,
  31,
  36,
  34,
  33,
  38,
  37,
  35,
  32,
  30,
  29,
  21,
  14,
  10,
  1],
 6708.082176454046)